# import Libraries

In [25]:
import pandas as pd
from pulp import * #to build and solve the BIP model

# Load the Dataset

Columns used (from paper concepts):

role → team composition

bat_avg, strike_rate → batting performance

bowl_avg, economy → bowling performance

fielding_score → fielding impact

price → budget constraint

foreign → overseas player constraint

In [26]:
data = pd.read_csv("player_stats_big.csv")

# Display first few rows
print(data.head())

    player          role  bat_avg  strike_rate  bowl_avg  economy  \
0  Player1       Batsman       43          114         0      0.0   
1  Player2       Batsman       33          122         0      0.0   
2  Player3        Bowler       12          102        19      8.0   
3  Player4  WicketKeeper       47          111        29      5.6   
4  Player5       Batsman       30          114         0      0.0   

   fielding_score  price  foreign  
0               4      9        0  
1               4     12        1  
2               7      7        0  
3               4     12        0  
4               7      4        0  


# Define the Decision Variables (xi)
“Each player is represented by a binary decision variable.”
xi = 1 → player i is selected
xi = 0 → player i is not selected


In [27]:
players = data["player"].tolist()

# Binary decision variables
x = LpVariable.dicts("select", players, cat="Binary")


# Define the Objective Function
“Maximize total team performance based on weighted performance indices.”

We compute a performance score exactly like a weighted index in the paper.

# formula:
Score =
0.4 × Batting Average
+ 0.3 × Strike Rate
− 0.2 × Bowling Average
− 0.2 × Economy
+ 0.3 × Fielding Score


In [28]:
# Calculate performance score for each player
data["score"] = (
    0.4 * data["bat_avg"]
    + 0.3 * data["strike_rate"]
    - 0.2 * data["bowl_avg"]
    - 0.2 * data["economy"]
    + 0.3 * data["fielding_score"]
)


In [29]:
# Create BIP model
model = LpProblem("Cricket_Team_Selection", LpMaximize)

# Objective function
model += lpSum(data.loc[i, "score"] * x[players[i]]
               for i in range(len(players)))


# Add Team Size Constraint
“The team must consist of exactly 11 players.”


∑ xi = 11


In [30]:
model += lpSum(x[p] for p in players) == 11


# Add Role-Based Constraints
At least 5 batsmen

At least 5 bowlers

Exactly 1 wicketkeeper

∑ xi (batsmen) ≥ 5

∑ xi (bowlers) ≥ 5

∑ xi (wicketkeeper) = 1


In [31]:
model += lpSum(
    x[data.loc[i, "player"]]
    for i in data[data["role"] == "Batsman"].index
) >= 5


In [32]:
model += lpSum(
    x[data.loc[i, "player"]]
    for i in data[data["role"] == "Bowler"].index
) >= 5


In [33]:
model += lpSum(
    x[data.loc[i, "player"]]
    for i in data[data["role"] == "WicketKeeper"].index
) == 1


# Add Budget Constraint

“Budget limitations can be incorporated as constraints.”

∑ (price × xi) ≤ Budget


In [34]:
BUDGET = 75

model += lpSum(
    data.loc[i, "price"] * x[players[i]]
    for i in range(len(players))
) <= BUDGET


# Add Foreign Player Constraint

“Foreign player restrictions can be included.”

∑ (foreign × xi) ≤ 4


In [35]:
model += lpSum(
    data.loc[i, "foreign"] * x[players[i]]
    for i in range(len(players))
) <= 4


# Solve the Optimization Model

In [36]:
model.solve()

print("Solver Status:", LpStatus[model.status])


Solver Status: Optimal


# Display the Selected Team

In [37]:
print("\n✅ Selected Playing XI:\n")

total_score = 0
total_budget = 0
foreign_count = 0

for i in range(len(players)):
    if x[players[i]].value() == 1:
        row = data.loc[i]
        print(f"{row['player']} | {row['role']} | Score: {row['score']:.2f}")
        total_score += row["score"]
        total_budget += row["price"]
        foreign_count += row["foreign"]

print("\nTotal Team Score:", round(total_score, 2))
print("Total Budget Used:", total_budget)
print("Foreign Players:", foreign_count)



✅ Selected Playing XI:

Player26 | Batsman | Score: 60.40
Player33 | Batsman | Score: 62.40
Player36 | Bowler | Score: 44.24
Player44 | Batsman | Score: 59.70
Player46 | Bowler | Score: 48.90
Player50 | Bowler | Score: 44.54
Player51 | Batsman | Score: 62.40
Player54 | Bowler | Score: 45.38
Player56 | Bowler | Score: 46.20
Player60 | Batsman | Score: 61.90
Player70 | WicketKeeper | Score: 53.00

Total Team Score: 589.06
Total Budget Used: 75
Foreign Players: 1


# if you need

In [38]:
import pandas as pd
import random

# -------------------------------
# Load Dataset
# -------------------------------
data = pd.read_csv("player_stats_big.csv")

# -------------------------------
# Separate Players by Role
# -------------------------------
batsmen = data[data["role"] == "Batsman"]
bowlers = data[data["role"] == "Bowler"]
keepers = data[data["role"] == "WicketKeeper"]
allrounders = data[data["role"] == "AllRounder"]

# -------------------------------
# Random Selection with Constraints
# -------------------------------
selected_batsmen = batsmen.sample(5)
selected_bowlers = bowlers.sample(5)
selected_keeper = keepers.sample(1)

# Remove already selected players
remaining = data.drop(
    selected_batsmen.index
).drop(
    selected_bowlers.index
).drop(
    selected_keeper.index
)

# Select 1 more player randomly
selected_extra = remaining.sample(1)

# Final Team
random_team = pd.concat([
    selected_batsmen,
    selected_bowlers,
    selected_keeper,
    selected_extra
])

# -------------------------------
# Display Result
# -------------------------------
print("🎲 RANDOMLY SELECTED PLAYING XI\n")

for _, row in random_team.iterrows():
    print(f"{row['player']} | {row['role']}")

print("\nTotal Players Selected:", len(random_team))


🎲 RANDOMLY SELECTED PLAYING XI

Player27 | Batsman
Player26 | Batsman
Player78 | Batsman
Player75 | Batsman
Player43 | Batsman
Player42 | Bowler
Player48 | Bowler
Player3 | Bowler
Player65 | Bowler
Player34 | Bowler
Player6 | WicketKeeper
Player67 | Bowler

Total Players Selected: 12
